# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [35]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,new plymouth,-39.0667,174.0833,11.00,87,14,6.18,NZ,1689538153
1,1,aginskoye,51.1000,114.5300,16.70,68,10,1.23,RU,1689538508
2,2,adamstown,-25.0660,-130.1015,20.71,79,100,3.39,PN,1689538508
3,3,alaghsas,17.0187,8.0168,37.03,16,1,4.46,NE,1689538509
4,4,port-aux-francais,-49.3500,70.2167,1.97,54,96,12.79,TF,1689538509


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [53]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng", "Lat",  geo = True, tiles = "OSM", size = "Humidity",  color = "City", frame_width=700, frame_height=500)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) 
                    & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
93,93,urumqi,43.8010,87.6005,26.53,34,0,3.00,CN,1689538554
168,168,arraial do cabo,-22.9661,-42.0278,23.98,78,0,2.57,BR,1689538593
249,249,kietrz,50.0804,18.0043,22.28,89,0,1.33,PL,1689538632
321,321,kavarna,43.4333,28.3333,23.10,82,0,3.34,BG,1689538668
378,378,glendive,47.1053,-104.7125,23.79,46,0,3.60,US,1689538696
395,395,mawu,29.5923,107.3139,21.14,89,0,1.74,CN,1689538705
452,452,chernomorskoye,45.5039,32.6997,23.74,56,0,2.35,UA,1689538732
459,459,darwin,-12.4611,130.8418,23.99,88,0,0.00,AU,1689538503
476,476,niimi,34.9833,133.4667,21.33,94,0,0.65,JP,1689538744
491,491,ogulin,45.2661,15.2286,26.24,69,0,1.98,HR,1689538752


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy(deep=True)
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
93,urumqi,CN,43.8010,87.6005,34,
168,arraial do cabo,BR,-22.9661,-42.0278,78,
249,kietrz,PL,50.0804,18.0043,89,
321,kavarna,BG,43.4333,28.3333,82,
378,glendive,US,47.1053,-104.7125,46,
395,mawu,CN,29.5923,107.3139,89,
452,chernomorskoye,UA,45.5039,32.6997,56,
459,darwin,AU,-12.4611,130.8418,88,
476,niimi,JP,34.9833,133.4667,94,
491,ogulin,HR,45.2661,15.2286,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
params = {
    "categories":"accommodation.hotel", "apiKey":geoapify_key, "limit":limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
  
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
urumqi - nearest hotel: 百时快捷酒店红山店
arraial do cabo - nearest hotel: No hotel found
kietrz - nearest hotel: Hotel Kombinat
kavarna - nearest hotel: Добротица
glendive - nearest hotel: La Quinta Inn & Suites by Wyndham Glendive
mawu - nearest hotel: No hotel found
chernomorskoye - nearest hotel: Море
darwin - nearest hotel: Mantra Pantanas Darwin
niimi - nearest hotel: No hotel found
ogulin - nearest hotel: Hotel Frankopan
baotou - nearest hotel: 包头宾馆


,City,Country,Lat,Lng,Humidity,Hotel Name
93,urumqi,CN,43.8010,87.6005,34,百时快捷酒店红山店
168,arraial do cabo,BR,-22.9661,-42.0278,78,No hotel found
249,kietrz,PL,50.0804,18.0043,89,Hotel Kombinat
321,kavarna,BG,43.4333,28.3333,82,Добротица
378,glendive,US,47.1053,-104.7125,46,La Quinta Inn & Suites by Wyndham Glendive
395,mawu,CN,29.5923,107.3139,89,No hotel found
452,chernomorskoye,UA,45.5039,32.6997,56,Море
459,darwin,AU,-12.4611,130.8418,88,Mantra Pantanas Darwin
476,niimi,JP,34.9833,133.4667,94,No hotel found
491,ogulin,HR,45.2661,15.2286,69,Hotel Frankopan


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [52]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng", "Lat",  geo = True, tiles = "OSM", 
                                    size = "Humidity",  color = "City", hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)